In [23]:
import glob
import os
import pandas as pd

%matplotlib inline

from tqdm.notebook import tqdm
tqdm.pandas()

DROOT = './data'

/Users/ivo/opt/anaconda3/envs/thesis-env/lib/python3.7/site-packages/tqdm/std.py:668: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


https://osmcode.org/osmium-tool/manual.html

In [7]:
city_list_df = pd.read_csv(os.path.join(DROOT, '1-raw', 'citylist.csv'))
city_list_df.tail()

,City,Continent,Order,Country,ctr_name,ctr_lat,ctr_lon,cellsanalysed
54,São Paulo,SAM,3,BRA,Edifício Matarazzo,-23.547501,-46.637259,5211.0
55,Rio de Janeiro,SAM,3,BRA,Câmara Municipal do Rio de Janeiro,-22.909969,-43.176650,2007.0
56,Salvador,SAM,3,BRA,Câmara Municipal de Salvador,-12.974940,-38.512481,1756.0
57,Santiago,SAM,3,CHL,Congreso Nacional de Chile sede Santiago,-33.438357,-70.653394,4962.0
58,Bogota,SAM,3,COL,Palacio de lievano,4.598538,-74.076741,3192.0


In [9]:
out_dict={}
for city in tqdm(city_list_df.itertuples(), total=len(city_list_df)):
    gdf_pop = pd.read_pickle(
        os.path.join(DROOT, '3-interim', 'populationmasks', city.City + '.pcl'))
    out_dict[city.City] = {
        'continent': city.Continent,
        'bbox': gdf_pop.total_bounds
    }


In [22]:
docker_root = '../../openrouteservice/docker/'
# Translate continent to the osm-file.
con2osm = {
    'EUR': 'europe-latest',
    'OCE': 'australia-oceania-latest',
    'ASI': 'asia-latest',
    'NAM': 'north-america-latest',
    'SAM': 'south-america-latest',
}

# Generate OSM extract for every item.
for city, props in tqdm(out_dict.items()):

    srcp = os.path.join(DROOT, '2-external', 'osmpbf', 
                        con2osm[props['continent']] + '.osm.pbf')
    dstp = os.path.join(docker_root, 'data', f'{city}.osm.gz')
    
    # Skip if already generated.
    if os.path.isfile(dstp):
        print("- {} already generated".format(city))
        continue
        
    bbox = ','.join([str(b) for b in props['bbox']])

    # Add asterisks for using in osmium
    dstp = f'"{dstp}"'
    print(city + ': ' + dstp)
    !osmium extract --bbox $bbox $srcp --output $dstp

- Vienna already generated
- Graz already generated
- Brussels already generated
- Berlin already generated
- Hamburg already generated
- Copenhagen already generated
- Aarhus already generated
- Madrid already generated
- Barcelona already generated
- Paris already generated
- Marseille already generated
- London already generated
- Birmingham already generated
- York already generated
- Luxembourg already generated
- Amsterdam already generated
- Utrecht already generated
Milan: "../../openrouteservice/docker/data/Milan.osm.gz"
[======================================================================] 100% >                                ]  53% 
Rome: "../../openrouteservice/docker/data/Rome.osm.gz"
[======================================================================] 100% 
- Stockholm already generated
- Gothenburg already generated
- Helsinki already generated
- Oslo already generated
- Warsaw already generated
- Lisbon already generated
- Bern already generated
- Prague already 